# Introduction

Scipp can handle *event data*, a certain type of sparse data, i.e., data that cannot directly be represented as a multi-dimensional array.
For applications that rely solely on dense arrays of data this section can safely be ignored.

Scipp supports event data in shape of a multi-dimensional array of lists.
This could, e.g., be used to store data from an array of sensors/detectors that are read out independently, with potentially widely varying frequency.

Event data can be created using a special `dtype`, `event_list_float64`, `event_list_float32`, `event_list_int64`, and `event_list_int32`.
Since here we are not dealing with a dense array we cannot set values for all `x` from a numpy array.
The recommended approach is to slice out all dimensions.
Then the remaining values (for a particluar "x" in this case) are a dense array with a list-like interface.
Initially all lists are empty:

In [ ]:
import numpy as np
import scipp as sc
from scipp.plot import plot

var = sc.Variable(dims=['x'],
                  shape=[4],
                  dtype=sc.dtype.event_list_float64)
sc.show(var)
var

In [ ]:
var['x', 0].values = np.arange(3)
var['x', 1].values.append(42)
var['x', 0].values.extend(np.ones(3))
var['x', 3].values = np.ones(6)
sc.show(var)
var

In [ ]:
var['x', 0].values

In [ ]:
var['x', 1].values

In [ ]:
var['x', 2].values

Operations between variables or datasets broadcast dense data to lists:

In [ ]:
scale = sc.Variable(dims=['x'], values=np.arange(2.0, 6))
var *= scale
var['x', 0].values

In [ ]:
var['x', 1].values

In [ ]:
var['x', 2].values

Event data in a data array has values corresponding to "weights" of events.
If each event corresponds to, e.g., a single detected neutron the weight is 1.
Event data in a data array is be associated with a corresponding event coordinate:

In [ ]:
weights = sc.Variable(
    dims=['x'],
    shape=[4],
    unit=sc.units.counts,
    dtype=sc.dtype.event_list_float64,
    variances=True)
weights['x', 0].values = np.ones(6)
weights['x', 1].values = np.ones(1)
weights['x', 2].values = np.ones(0)
weights['x', 3].values = np.ones(6)
weights['x', 0].variances = np.ones(6)
weights['x', 1].variances = np.ones(1)
weights['x', 2].variances = np.ones(0)
weights['x', 3].variances = np.ones(6)

a = sc.DataArray(
    data=weights,
    coords={'x': sc.Variable(['x'], values=np.arange(4.0)),
            'time': var})
a.coords['time'].unit = sc.units.us # micro second
a

The lengths of the sublists between coordinate and values (and variances) must match.
Scipp does not enforce this when modifying sublists, but *does* verify correctness in operations on variables or data arrays.

In the graphical representation of the data array we can see the event coordinate (green), and the event values and variances (yellow):

In [ ]:
sc.show(a)

# Arithmetic operations

Direct arithmetic operations with event data can be useful in some cases, but more commonly operations that act on event data as if it had been histogrammed are required.
For example, addition of histogrammed data would correspond to concatenating event lists.

Scipp supports such operations based on realigned wrappers of event data.
For an introduction of the concept of realigned data see [Unaligned and Realigned Data](../user-guide/unaligned-data.ipynb).
The following operations are supported:

- Addition of data arrays containing realigned event data.
  Internally this concatenates the underlying event lists.
- Subtraction of data arrays containing realigned event data.
  Internally this concatenates the underlying event lists with a negative weight for the events in the subtrahend.
- Multiplication of a data array containing realigned event data with a data array with dense, histogrammed data.
  The weight of each event is scaled by the value of the corresponding bin in the histogram.
- Division of a data array containing realigned event data by a data array with dense, histogrammed data.
  The weight of each event is divided by the value of the corresponding bin in the histogram.

<div class="alert alert-warning">
    <b>WARNING:</b>

It is important to note that these operations, in particular multiplication and division, are only interchangeable with histogramming if the variances of the "histogram" operand are negligible.
If these variances are not negligible the operation on the event data introduces correlations in the error bars of the individual events.
Scipp has no way of tracking such correlations and a subsequent `histogram` step propagates uncertainties under the assumption of uncorrelated error bars.
</div>

We use `realign` to wrap the event data with a time dimension:

In [ ]:
time_bins = sc.Variable(dims=['time'], unit=sc.units.us, values=[0.0, 3.0, 6.0])
realigned = sc.realign(a, {'time':time_bins})
realigned

## Addition

In [ ]:
realigned += realigned
sc.show(realigned.unaligned)
plot(realigned)

## Subtraction

In [ ]:
zero = realigned.copy()
zero -= zero
sc.show(zero.unaligned)
plot(zero)

## Multiplication and division

In [ ]:
realigned /= sc.histogram(realigned)
sc.show(realigned.unaligned)
plot(realigned)